# **Neural Network for Image Classification**
*   Implementation of a classic neural network for image classification supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a classic neural network for image classification supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementations are based off of TensorFlow's very own example: [TensorFlow Image Claasification](https://www.tensorflow.org/tutorials/keras/classification)

In [1]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

In [2]:
# Set a global TF random seed
tf.random.set_seed(12)

In [3]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 3
# Values are specific to the Fashion MNIST dataset
INPUT_DIM = 28
NUM_CLASSES = 10

In [4]:
def build_and_train(X_train, y_train, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Flatten(input_shape=(INPUT_DIM, INPUT_DIM), dtype=dtype), # !!!
        layers.Dense(128, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [5]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    model = models.Sequential([
        layers.Flatten(input_shape=(INPUT_DIM, INPUT_DIM)),
        layers.Dense(128, activation='relu'),
        layers.Dense(NUM_CLASSES)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [6]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test  = X_test / 255.0

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4950 - accuracy: 0.8258
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3735 - accuracy: 0.8663
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3352 - accuracy: 0.8777
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3129 - accuracy: 0.8847
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2937 - accuracy: 0.8922
Epoch 1/5
1875/1875 [==============================] - 8s 3ms/step - loss: 0.4959 - accuracy: 0.8255
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3760 - accuracy: 0.8657
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3356 - accuracy: 0.8785
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3122 - accuracy: 0.8851
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2953 - accurac

In [8]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4965 - accuracy: 0.8256
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3735 - accuracy: 0.8655
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3340 - accuracy: 0.8778
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3123 - accuracy: 0.8849
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2944 - accuracy: 0.8915
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4984 - accuracy: 0.8244
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3792 - accuracy: 0.8634
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3396 - accuracy: 0.8757
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3163 - accuracy: 0.8838
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2968 - accurac

In [9]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4916 - accuracy: 0.8274
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3699 - accuracy: 0.8668
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3320 - accuracy: 0.8790
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3095 - accuracy: 0.8857
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2921 - accuracy: 0.8917
Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4963 - accuracy: 0.8252
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3735 - accuracy: 0.8658
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3346 - accuracy: 0.8779
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3107 - accuracy: 0.8853
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2936 - accurac

In [10]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 6s 2ms/step - loss: 16.5092 - accuracy: 0.6648
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 5.0583 - accuracy: 0.7741
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 3.6915 - accuracy: 0.7963
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 3.0713 - accuracy: 0.8069
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.6068 - accuracy: 0.8135
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 12.0793 - accuracy: 0.6810
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 3.8466 - accuracy: 0.7767
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.7524 - accuracy: 0.7949
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.1667 - accuracy: 0.8081
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.1173 - accur

In [13]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4927 - accuracy: 0.8266
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3714 - accuracy: 0.8662
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3332 - accuracy: 0.8783
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3099 - accuracy: 0.8855
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2918 - accuracy: 0.8917
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4936 - accuracy: 0.8252
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3720 - accuracy: 0.8661
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3339 - accuracy: 0.8776
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3104 - accuracy: 0.8847
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2928 - accurac

In [14]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 38.435929695765175 seconds
Average training time in single precision: 30.805833260218304 seconds
Average training time in half precision: 36.61778155962626 seconds
Average training time in mixed half precision: 34.083193937937416 seconds
-------------
Accuracy with double precision: 0.8723999857902527
Accuracy with single precision: 0.8707000017166138
Accuracy with half precision: 0.808899998664856
Accuracy with mixed half precision: 0.8727999925613403
